## Descriptions:
WE DON'T USE SIMPLE DATA IN THIS PROJECT -- can disregard this notebook

Combine all features with vital summary and labels to create the simple dataset if needed

**Inputs**:  
- `6_5_coh3_vital_stats` contains all vital related features
- `6_8_coh3_imputedHWESI` contains all demo, imputed HW and ESI
- merge with `6_7_cohort4` -- cohort with labels (and labs) to be consistent with the complex data

**Outputs**: 
- `6_8_coh4_simpledata`: in *wide* format

### Importing R libraries

In [1]:
library(bigrquery)  # to query STARR-OMOP (stored in BigQuery) using SQL
library(tidyverse)
library(lubridate)
# library(mice)
# library(VIM) # for missing data plot

# library(data.table)
# library(Matrix)
# library(caret) # import this before glmnet to avoid rlang version problem
# library(glmnet)
# library(bit64)

# library(slam)
# library(mtools) for one hot coder, not available on Nero or use caret or tidyr
options(repr.matrix.max.rows=250, repr.matrix.max.cols=30)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




### Get the dataset for simple models:

In [4]:
# read inputs
datadir6 = "../../DataTD/validation"
valdir = "../../OutputTD/6_validation"

vital_stats <- read.csv(file.path(valdir, '6_5_coh3_vital_stats.csv'))
nrow(vital_stats) #44258

cohort_imp <- read.csv(file.path(valdir, '6_8_coh4_all_imputedHWESI.csv'))
nrow(cohort_imp) #44258

cohort <- read.csv(file.path(valdir, '6_7_cohort4.csv')) #41366
nrow(cohort)

[1] 16704

[1] 60464

[1] 16484

In [ ]:
sum(!vital_stats$pat_enc_csn_id_coded %in% cohort_imp$pat_enc_csn_id_coded)
length(setdiff(cohort$pat_enc_csn_id_coded, cohort_imp$pat_enc_csn_id_coded))
length(setdiff(cohort$pat_enc_csn_id_coded, vital_stats$pat_enc_csn_id_coded))

head(vital_stats, n=1)
head(cohort_imp, n=1)
head(cohort, n=1)

In [ ]:
vital_stats <- vital_stats %>% #mutate(admit_time = ymd_hms(admit_time), label_max24 = label) %>%
                select(-inpatient_data_id_coded, -admit_time, -label)
cohort_imp <- cohort_imp %>% #mutate(admit_time = ymd_hms(admit_time), label_max24 = label) %>%
                 select(-inpatient_data_id_coded, -admit_time, -label)
# cohort <- cohort %>% mutate(admit_time = ymd_hms(admit_time))

In [ ]:
data_simple <- cohort %>% select(anon_id, pat_enc_csn_id_coded, first_label, death_24hr_recent_label) %>%
                left_join(cohort_imp) %>% left_join(vital_stats) # left_join both, otherwise if full_join, it will retain all rows in vitals
dim(data_simple)
nrow(data_simple %>% select(pat_enc_csn_id_coded) %>% distinct())
colnames(data_simple) #41366

In [ ]:
summary(data_simple)

In [ ]:
# write.csv(cohort_final, file.path(datadir, 'cohort_final.csv'), row.names = FALSE)
write.csv(data_simple, file.path(valdir, "6_8_coh4_simpledata.csv"), row.names=FALSE)